# Convert XArray to Pickled Dictionary
Opens data from solar irradiance and cloud coverage files with xarray, does a little preprocessing, and converts to a pickled dictionary for use as dummy data for the plugin

In [1]:
import pickle
import xarray as xr

In [22]:
def open_file_with_xr(path):
    data = xr.open_mfdataset(path)
    return data

In [23]:
# structured as: {name of variable in app.py: [file path, name of variable we want from that file]}
paths = {"env.solar.irradiance": ['sgpradflux1longC1.c2.20160301.060000.nc', 'downwelling_shortwave'],
         "env.coverage.cloud": ['sgptsiskycoverC1.b1.20160301.000000.cdf', 'percent_opaque']}

# create a dicitonary of xr datasets for each variable
xr_dict = {}
for name, xr_details in paths.items():
    path = xr_details[0]
    var_name = xr_details[1]
    data = open_file_with_xr(path)[var_name]
    xr_dict[name] = data

# get the intersection of each variable's dataset (intersection of time points)
merged = xr.merge(xr_dict.values(), join='inner')

# now take each variable's data and put it back into a dictionary for pickling
data_dict = {}
for name, xr_details in paths.items():
    var_name = xr_details[1]
    data_dict[name] = merged[var_name].values

# pickle the data dict
with open('20160301_ex_data.pickle', 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)